In [1]:
import pandas as pd

In [2]:
orderdf = pd.read_csv('./01Feb2021/NIFTY21APRFUT_1.CSV')

In [3]:
#https://www.youtube.com/watch?v=oqUR41U-xT4&list=WL&index=11

In [4]:
orderdf.columns

Index(['Sqn', 'Timestamp', 'TIME', 'Ord_ID1', 'Ord_ID2', 'Price', 'Qty',
       'MessageType', 'LTP', 'LTQ', 'VTT', 'ORDER TYPE', 'BIDQ1', 'BID1',
       'ASK1', 'ASKQ1', 'BIDQ2', 'BID2', 'ASK2', 'ASKQ2', 'BIDQ3', 'BID3',
       'ASK3', 'ASKQ3', 'BIDQ4', 'BID4', 'ASK4', 'ASKQ4', 'BIDQ5', 'BID5',
       'ASK5', 'ASKQ5'],
      dtype='object')

In [5]:
orderdf.columns = ['Timestamp', 'TIME', 'Ord_ID1', 'Ord_ID2', 'Price', 'Qty',\
       'MessageType', 'LTP', 'LTQ', 'VTT', 'ORDER TYPE', 'BIDQ1', 'BID1',\
       'ASK1', 'ASKQ1', 'BIDQ2', 'BID2', 'ASK2', 'ASKQ2', 'BIDQ3', 'BID3',\
       'ASK3', 'ASKQ3', 'BIDQ4', 'BID4', 'ASK4', 'ASKQ4', 'BIDQ5', 'BID5',\
       'ASK5', 'ASKQ5', 'last']

In [6]:
orderdf.index.name = 'Sqn'

In [7]:
orderdf.head()

,Timestamp,TIME,Ord_ID1,Ord_ID2,Price,Qty,MessageType,LTP,LTQ,VTT,...,ASKQ3,BIDQ4,BID4,ASK4,ASKQ4,BIDQ5,BID5,ASK5,ASKQ5,last
Sqn,,,,,,,,,,,,,,,,,,,,,
7,1296638100009989966,09:15:00.009989,1100000000000916,0,13745.10,3000,NEW ORDER,0.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
511,1296638100018450022,09:15:00.018450,1100000000001356,0,13929.85,150,NEW ORDER,0.0,0,0,...,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
621,1296638100022986851,09:15:00.022986,1100000000001473,0,13745.15,150,NEW ORDER,0.0,0,0,...,600.0,3000.0,13515.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
872,1296638100025936587,09:15:00.025936,1100000000001631,0,13929.80,75,NEW ORDER,0.0,0,0,...,3000.0,3000.0,13515.3,14149.9,600.0,NaN,NaN,NaN,NaN,NaN
873,1296638100025940922,09:15:00.025940,1100000000001632,0,13929.80,75,NEW ORDER,0.0,0,0,...,3000.0,3000.0,13515.3,14149.9,600.0,NaN,NaN,NaN,NaN,NaN


In [8]:
#orderdf.info()

In [9]:
#orderdf['TIME2'] = pd.to_datetime(orderdf['TIME'], infer_datetime_format= True)
#orderdf = orderdf.drop(['TIME2'], axis=1)
#from datetime import datetime
#datetime.now()

In [10]:
pd.to_datetime(orderdf['TIME'], format='%H:%M:%S.%f')

Sqn
7          1900-01-01 09:15:00.009989
511        1900-01-01 09:15:00.018450
621        1900-01-01 09:15:00.022986
872        1900-01-01 09:15:00.025936
873        1900-01-01 09:15:00.025940
                      ...            
30072612   1900-01-01 09:29:59.867681
30072613   1900-01-01 09:29:59.867683
30072614   1900-01-01 09:29:59.867686
30072627   1900-01-01 09:29:59.867824
30075090   1900-01-01 09:29:59.930111
Name: TIME, Length: 132725, dtype: datetime64[ns]

In [11]:
orderdf['TIME'] = pd.to_datetime(orderdf['TIME'], format='%H:%M:%S.%f')

In [12]:
orderdf = orderdf.set_index(['TIME'])

In [13]:
orderdf

,Timestamp,Ord_ID1,Ord_ID2,Price,Qty,MessageType,LTP,LTQ,VTT,ORDER TYPE,...,ASKQ3,BIDQ4,BID4,ASK4,ASKQ4,BIDQ5,BID5,ASK5,ASKQ5,last
TIME,,,,,,,,,,,,,,,,,,,,,
1900-01-01 09:15:00.009989,1296638100009989966,1100000000000916,0,13745.10,3000,NEW ORDER,0.0,0,0,BUY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1900-01-01 09:15:00.018450,1296638100018450022,1100000000001356,0,13929.85,150,NEW ORDER,0.0,0,0,SELL,...,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1900-01-01 09:15:00.022986,1296638100022986851,1100000000001473,0,13745.15,150,NEW ORDER,0.0,0,0,BUY,...,600.0,3000.0,13515.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1900-01-01 09:15:00.025936,1296638100025936587,1100000000001631,0,13929.80,75,NEW ORDER,0.0,0,0,SELL,...,3000.0,3000.0,13515.30,14149.90,600.0,NaN,NaN,NaN,NaN,NaN
1900-01-01 09:15:00.025940,1296638100025940922,1100000000001632,0,13929.80,75,NEW ORDER,0.0,0,0,SELL,...,3000.0,3000.0,13515.30,14149.90,600.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1900-01-01 09:29:59.867681,1296638999867681102,1100000002810135,0,13801.00,75,CANCELLATION,13801.6,75,5175,SELL,...,75.0,1650.0,13782.95,13801.05,75.0,75.0,13782.25,13801.45,75.0,NaN
1900-01-01 09:29:59.867683,1296638999867683566,1100000002810136,0,13792.80,75,CANCELLATION,13801.6,75,5175,BUY,...,75.0,75.0,13782.25,13801.05,75.0,750.0,13781.60,13801.45,75.0,NaN
1900-01-01 09:29:59.867686,1296638999867686981,1100000002810139,0,13801.05,75,CANCELLATION,13801.6,75,5175,SELL,...,75.0,75.0,13782.25,13801.45,75.0,750.0,13781.60,13803.05,75.0,NaN


**PTR**: Consider rectifying the date as 01-01-1900. This needs to be corrected. I am trying to find a way to specify the exact date. 

In [14]:
price_ohlc = orderdf['Price'].resample('60s').ohlc()
price_ohlc

,open,high,low,close
TIME,,,,
1900-01-01 09:15:00,13745.10,14084.70,13594.35,13822.30
1900-01-01 09:16:00,13822.30,13850.45,13780.25,13812.90
1900-01-01 09:17:00,13827.10,13849.80,13800.30,13831.25
1900-01-01 09:18:00,13841.00,13853.45,13809.10,13832.85
1900-01-01 09:19:00,13841.25,13850.80,13797.35,13818.55
1900-01-01 09:20:00,13810.40,13823.55,13761.35,13800.35
1900-01-01 09:21:00,13800.30,13813.40,13763.00,13787.05
1900-01-01 09:22:00,13793.95,13830.45,13774.60,13798.45
1900-01-01 09:23:00,13813.40,13840.00,13788.90,13817.45


In [15]:
"""
bids = orderdf['BID1'].resample('60s').ohlc()
bids

asks = orderdf['ASK2'].resample('60s').ohlc()
asks
"""

"\nbids = orderdf['BID1'].resample('60s').ohlc()\nbids\n\nasks = orderdf['ASK2'].resample('60s').ohlc()\nasks\n"

### Total successful trades

In [16]:
orderdf.MessageType.unique()

array(['NEW ORDER', 'MODIFICATION', 'CANCELLATION', 'TRADE'], dtype=object)

In [17]:
tradedf = orderdf[orderdf.MessageType == 'TRADE']

In [19]:
successful_trades = tradedf['MessageType'].resample('1min').count()
successful_trades

TIME
1900-01-01 09:15:00    26
1900-01-01 09:16:00     6
1900-01-01 09:17:00     5
1900-01-01 09:18:00     2
1900-01-01 09:19:00     4
1900-01-01 09:20:00     6
1900-01-01 09:21:00     3
1900-01-01 09:22:00     1
1900-01-01 09:23:00     5
1900-01-01 09:24:00     2
1900-01-01 09:25:00     2
1900-01-01 09:26:00     0
1900-01-01 09:27:00     1
1900-01-01 09:28:00     4
1900-01-01 09:29:00     1
Freq: T, Name: MessageType, dtype: int64

In [20]:
cancelled_trades = orderdf[orderdf.MessageType == 'CANCELLATION']['MessageType'].resample('1min').count()
cancelled_trades

TIME
1900-01-01 09:15:00    1067
1900-01-01 09:16:00    1250
1900-01-01 09:17:00     949
1900-01-01 09:18:00    1316
1900-01-01 09:19:00    1330
1900-01-01 09:20:00    2727
1900-01-01 09:21:00    1790
1900-01-01 09:22:00    2731
1900-01-01 09:23:00    2576
1900-01-01 09:24:00    1894
1900-01-01 09:25:00    1826
1900-01-01 09:26:00    1685
1900-01-01 09:27:00    1449
1900-01-01 09:28:00    1087
1900-01-01 09:29:00    1263
Freq: T, Name: MessageType, dtype: int64

In [21]:
new_orders = orderdf[orderdf.MessageType == 'NEW ORDER']['MessageType'].resample('1min').count()
new_orders

TIME
1900-01-01 09:15:00    1127
1900-01-01 09:16:00    1258
1900-01-01 09:17:00     947
1900-01-01 09:18:00    1336
1900-01-01 09:19:00    1330
1900-01-01 09:20:00    2760
1900-01-01 09:21:00    1838
1900-01-01 09:22:00    2756
1900-01-01 09:23:00    2589
1900-01-01 09:24:00    1938
1900-01-01 09:25:00    1812
1900-01-01 09:26:00    1674
1900-01-01 09:27:00    1441
1900-01-01 09:28:00    1109
1900-01-01 09:29:00    1254
Freq: T, Name: MessageType, dtype: int64

In [22]:
modified_df = orderdf[orderdf.MessageType == 'MODIFICATION']['MessageType'].resample('1min').count()
modified_df

TIME
1900-01-01 09:15:00    9001
1900-01-01 09:16:00    4286
1900-01-01 09:17:00    3394
1900-01-01 09:18:00    3725
1900-01-01 09:19:00    4317
1900-01-01 09:20:00    7769
1900-01-01 09:21:00    5909
1900-01-01 09:22:00    6249
1900-01-01 09:23:00    8233
1900-01-01 09:24:00    5203
1900-01-01 09:25:00    4660
1900-01-01 09:26:00    5811
1900-01-01 09:27:00    6037
1900-01-01 09:28:00    4432
1900-01-01 09:29:00    3522
Freq: T, Name: MessageType, dtype: int64

In [23]:
df = pd.DataFrame()

In [24]:
df = pd.concat([df, new_orders, cancelled_trades, successful_trades, modified_df], axis=1)

In [25]:
df.columns = ['neworders', 'cancelled_orders', 'trades', 'modified']
df

,neworders,cancelled_orders,trades,modified
1900-01-01 09:15:00,1127,1067,26,9001
1900-01-01 09:16:00,1258,1250,6,4286
1900-01-01 09:17:00,947,949,5,3394
1900-01-01 09:18:00,1336,1316,2,3725
1900-01-01 09:19:00,1330,1330,4,4317
1900-01-01 09:20:00,2760,2727,6,7769
1900-01-01 09:21:00,1838,1790,3,5909
1900-01-01 09:22:00,2756,2731,1,6249
1900-01-01 09:23:00,2589,2576,5,8233
1900-01-01 09:24:00,1938,1894,2,5203


### Market Orders
This can be detected by comparing price with the Ask. If price == Ask then it's a market order. 

In [ ]:
(orderdf['ORDER TYPE'] == 'SELL' and orderdf['Price'] == orderdf['BID1'])

market_orders = orderdf[orderdf['ORDER TYPE'] == 'BUY' and orderdf['Price'] == orderdf['ASK1']]
market_orders

df['mynumbers'].apply(lambda x: 'True' if x <= 53 else 'False')

In [76]:
(orderdf['ORDER TYPE'] == 'BUY' )| (orderdf['Price'] == orderdf['ASK1'])

TIME
1900-01-01 09:15:00.009989     True
1900-01-01 09:15:00.018450     True
1900-01-01 09:15:00.022986     True
1900-01-01 09:15:00.025936     True
1900-01-01 09:15:00.025940     True
                              ...  
1900-01-01 09:29:59.867681    False
1900-01-01 09:29:59.867683     True
1900-01-01 09:29:59.867686    False
1900-01-01 09:29:59.867824     True
1900-01-01 09:29:59.930111     True
Length: 132725, dtype: bool

In [26]:
market_orders = orderdf[((orderdf['ORDER TYPE'] == 'BUY') & (orderdf['Price'] == orderdf['ASK1'])) \
                        | ((orderdf['ORDER TYPE'] == 'SELL') & (orderdf['Price'] == orderdf['BID1']))]

In [47]:
market_orders_df = market_orders['ORDER TYPE'].resample('1min').count()

In [45]:
market_orders_df.columns

['n_market_orders']

In [48]:
df = df.join(market_orders_df)
df.rename(columns = {'ORDER TYPE': 'no_market'}, inplace = True)

In [49]:
df

,neworders,cancelled_orders,trades,modified,no_market
1900-01-01 09:15:00,1127,1067,26,9001,NaN
1900-01-01 09:16:00,1258,1250,6,4286,NaN
1900-01-01 09:17:00,947,949,5,3394,1.0
1900-01-01 09:18:00,1336,1316,2,3725,0.0
1900-01-01 09:19:00,1330,1330,4,4317,0.0
1900-01-01 09:20:00,2760,2727,6,7769,0.0
1900-01-01 09:21:00,1838,1790,3,5909,0.0
1900-01-01 09:22:00,2756,2731,1,6249,0.0
1900-01-01 09:23:00,2589,2576,5,8233,1.0
1900-01-01 09:24:00,1938,1894,2,5203,1.0
